In [16]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModel
from nltk.tokenize import word_tokenize

In [17]:
df = pd.read_csv('google_play_reviews_full.csv')
df.head(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,app_id
0,d4306902-8a3c-4223-bcdd-e82bc361592f,Anwarpasha Pasha,https://play-lh.googleusercontent.com/a/ACg8oc...,Good interactive platform,5,0,2.24.23.78,2024-12-05 22:05:43,NaN,NaN,2.24.23.78,com.whatsapp
1,2ebe2058-dd8d-4b7c-9d06-85ecfe25cd63,Anurag Prajapati,https://play-lh.googleusercontent.com/a-/ALV-U...,I 🙋 like it,4,0,2.24.22.78,2024-12-05 22:05:26,NaN,NaN,2.24.22.78,com.whatsapp
2,3ffb073b-af16-499c-8592-c088cb9e74c1,Posani Nadipolla,https://play-lh.googleusercontent.com/a/ACg8oc...,Super,5,0,NaN,2024-12-05 22:05:03,NaN,NaN,NaN,com.whatsapp
3,c2647e17-255b-4606-a4e3-6e6572713382,sohil shaikh,https://play-lh.googleusercontent.com/a/ACg8oc...,Do you want betar,1,0,2.24.23.78,2024-12-05 22:05:03,NaN,NaN,2.24.23.78,com.whatsapp
4,784d0fb1-5c5a-46f7-9cfa-16e9fefe13d2,Uppala Sukumari,https://play-lh.googleusercontent.com/a-/ALV-U...,Good qpp,5,0,2.24.19.86,2024-12-05 22:05:02,NaN,NaN,2.24.19.86,com.whatsapp


In [18]:
df = df.drop(columns = ['userImage', 'at', 'replyContent', 'repliedAt', 'reviewCreatedVersion', 'reviewId'])
df.head(5)

,userName,content,score,thumbsUpCount,appVersion,app_id
0,Anwarpasha Pasha,Good interactive platform,5,0,2.24.23.78,com.whatsapp
1,Anurag Prajapati,I 🙋 like it,4,0,2.24.22.78,com.whatsapp
2,Posani Nadipolla,Super,5,0,NaN,com.whatsapp
3,sohil shaikh,Do you want betar,1,0,2.24.23.78,com.whatsapp
4,Uppala Sukumari,Good qpp,5,0,2.24.19.86,com.whatsapp


In [19]:
df.drop_duplicates(subset= 'content')

,userName,content,score,thumbsUpCount,appVersion,app_id
0,Anwarpasha Pasha,Good interactive platform,5,0,2.24.23.78,com.whatsapp
1,Anurag Prajapati,I 🙋 like it,4,0,2.24.22.78,com.whatsapp
2,Posani Nadipolla,Super,5,0,NaN,com.whatsapp
3,sohil shaikh,Do you want betar,1,0,2.24.23.78,com.whatsapp
4,Uppala Sukumari,Good qpp,5,0,2.24.19.86,com.whatsapp
...,...,...,...,...,...,...
12494,Lawrence Igbinovia,No free pass,1,0,NaN,com.crunchyroll.crunchyroid
12495,zachary delapaz,Why watchlist ia disappearing is they kept rem...,1,0,3.70.4,com.crunchyroll.crunchyroid
12496,Dj,nice dubing,5,0,3.70.4,com.crunchyroll.crunchyroid
12498,Ganesh Rahate,I like it but can you please add dragon Ball also,5,0,NaN,com.crunchyroll.crunchyroid


In [20]:
df = df.dropna(subset = ['content'])

In [21]:
import re

def clean_text(text):
    text = text.lower() 
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'[^\w\s]', '', text) 
    text = re.sub(r'\s+', ' ', text).strip() 
    return text

df['content_cleaned'] = df['content'].apply(clean_text)

In [22]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered)
df['content_cleaned'] = df['content_cleaned'].apply(remove_stopwords)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/kietto03/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [33]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

df['content_cleaned'] = df['content_cleaned'].apply(lemmatize_text)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/kietto03/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
def categorize_rating(score):
    if score >= 4:
        return "positive"
    elif score == 3:
        return "neutral"
    else:
        return "negative"

df['sentiment'] = df['score'].apply(categorize_rating)

In [ ]:
# Split reviews based on sentiment
positive_reviews = df[df['sentiment'] == 'positive']
negative_reviews = df[df['sentiment'] == 'negative']
neutral_reviews = df[df['sentiment'] == 'neutral']

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [ ]:
import nltk
nltk.download()

: 